In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/Users/chrisonian/Code/GitHub/lume-gpt')

In [ ]:
import tempfile
import os
import h5py
from time import time
os.getcwd()

In [ ]:
from gpt import parsers, tools, writers

In [ ]:
class GPT:
    """ 
    GPT simulation object. Essential methods:
    .__init__(...)
    .configure()
    .run()
    
    Input deck is held in .input
    Output data is parsed into .output
    .load_screens() will load particle data into .screen[...]
    
    The GPT binary file can be set on init. If it doesn't exist, configure will check the
        $GPT_BIN
    environmental variable.
    
    
    """
    
    def __init__(self,
                 input_file=None,
                 gpt_bin='$GPT_BIN',      
                 use_tempdir=True,
                 workdir=None,
                 verbose=False):
        # Save init
        self.original_input_file = input_file
        self.use_tempdir = use_tempdir
        self.workdir = workdir
        if workdir:
            assert os.path.exists(workdir), 'workdir does not exist: '+workdir        
        self.verbose=verbose
        self.gpt_bin = gpt_bin

        # These will be set
        self.log = []
        self.output = {}
        self.screen = [] # list of screens
        self.timeout=None
        self.error = False
        
        # Run control
        self.finished = False
        self.configured = False
        self.using_tempdir = False
        
        # Call configure
        if input_file:
            self.load_input(input_file)
            self.configure()
        else:
            self.vprint('Warning: Input file does not exist. Not configured.')


                   

    def configure(self):
        self.configure_gpt(workdir=self.workdir)
 
    def configure_gpt(self, input_filePath=None, workdir=None):
        
        if input_filePath:
            self.load_input(input_filePath)
        
        # Check that binary exists
        self.gpt_bin = tools.full_path(self.gpt_bin)
        assert os.path.exists(self.gpt_bin), 'ERROR: GPT binary does not exist:'+self.gpt_bin
              
        # Set paths
        if self.use_tempdir:
            # Need to attach this to the object. Otherwise it will go out of scope.
            self.tempdir = tempfile.TemporaryDirectory(dir=workdir)
            self.path = self.tempdir.name
        else:
            # Work in place
            self.path = self.original_path            

        self.input_file = os.path.join(self.path, self.original_input_file)                
        self.configured = True
    
    
    def load_input(self, input_filePath, absolute_paths=True):
        f = tools.full_path(input_filePath)
        self.original_path, self.original_input_file = os.path.split(f) # Get original path, filename
        
        #TODO
        #self.input = parsers.parse_astra_input_file(f)            
            
            
    def load_output(self):
        #TODO
        pass
                
    def load_screens(self, end_only=False):
        # Clear existing screens
        self.screen = []
        
        #TODO
        
        
        
    def run(self):
        if not self.configured:
            print('not configured to run')
            return
        self.run_gpt(verbose=self.verbose, timeout=self.timeout)
        
    
        
    def get_run_script(self, write_to_path=True):
        """
        Assembles the run script. Optionally writes a file 'run' with this line to path.
        """
        
        _, infile = os.path.split(self.input_file)
        
        runscript = [self.gpt_bin, infile]
            
        if write_to_path:
            with open(os.path.join(self.path, 'run'), 'w') as f:
                f.write(' '.join(runscript))
            
        return runscript

        
    
    def run_gpt(self, verbose=False, parse_output=True, timeout=None):

        
        run_info = {}
        t1 = time()
        run_info['start_time'] = t1
        
        # Move to local directory

        # Save init dir
        init_dir = os.getcwd()
        self.vprint('init dir: ', init_dir)
        
        os.chdir(self.path)
        # Debugging
        self.vprint('running GPT in '+os.getcwd())

        # Write input file from internal dict
        self.write_input_file()
        
        runscript = self.get_run_script()
    
        try:
            if timeout:
                res = tools.execute2(runscript, timeout=timeout)
                log = res['log']
                self.error = res['error']
                run_info['why_error'] = res['why_error']
                # Log file must have this to have finished properly
                if log.find('finished simulation') == -1:
                    run_info['error'] = True
                    run_info.update({'error': True, 'why_error': "Couldn't find finished simulation"})
    
            else:
                # Interactive output, for Jupyter
                log = []
                for path in tools.execute(runscript):
                    self.vprint(path, end="")
                    log.append(path)
    
            self.log = log
                    
            if parse_output:
                self.load_output()
        except Exception as ex:
            print('Run Aborted', ex)
            self.error = True
            run_info['why_error'] = str(ex)
        finally:
            run_info['run_time'] = time() - t1
            run_info['run_error'] = self.error
            
            # Add run_info
            self.output.update(run_info)
            
            # Return to init_dir
            os.chdir(init_dir)                        
        
        self.finished = True
    
    def fingerprint(self):
        """
        Data fingerprint using the input. 
        """
        return tools.fingerprint(self.input)
                
    def vprint(self, *args, **kwargs):
        # Verbose print
        if self.verbose:
            print(*args, **kwargs)    
    
    
    def write_input_file(self):
        #TODO
        #parsers.write_namelists(self.input, self.input_file)
        pass
        
    # h5 writing
    def write_input(self, h5):
        #TODO
        #writers.write_input_h5(h5, self.input)
        pass
    
    def write_output(self, h5):
        #TODO
        writers.write_output_h5(h5, self.output)
        
    def write_screens(self, h5):
        #writers.write_screens_h5(h5, self.screen)    
        pass
        
    def archive(self, h5=None):
        """
        Archive all data to an h5 handle or filename.
        
        If no file is given, a file based on the fingerprint will be created.
        
        """
        if not h5:
            h5 = 'gpt_'+self.fingerprint()+'.h5'
         
        if isinstance(h5, str):
            g = h5py.File(h5, 'w')
            self.vprint(f'Archiving to file {h5}')
        else:
            g = h5
        
        # All input
        self.write_input(g)

        # All output
        self.write_output(g)
            
        # Particles    
        self.write_screens(g)          
        
        return h5
        

In [ ]:
G = GPT()